In [50]:
import os
from google.cloud import bigquery

In [51]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "analytics-399003-e59dddbd35b5.json"

In [52]:
client = bigquery.Client()

In [53]:
sql_query = """SELECT * FROM `analytics-399003.analytics_406290682.events_20230918` LIMIT 3"""

In [54]:
job = client.query(sql_query)

In [55]:
for row in job.result():
    print(row)

Row(('20230918', 1695048275080431, 'sec20_timer', [{'key': 'page_title', 'value': {'string_value': 'Viktor Byskov - Viktor Byskov', 'int_value': None, 'float_value': None, 'double_value': None}}, {'key': 'ga_session_id', 'value': {'string_value': None, 'int_value': 1695048254, 'float_value': None, 'double_value': None}}, {'key': 'page_location', 'value': {'string_value': 'https://viktorbyskov.dk/', 'int_value': None, 'float_value': None, 'double_value': None}}, {'key': 'session_engaged', 'value': {'string_value': '1', 'int_value': None, 'float_value': None, 'double_value': None}}, {'key': 'engagement_time_msec', 'value': {'string_value': None, 'int_value': 1591, 'float_value': None, 'double_value': None}}, {'key': 'ga_session_number', 'value': {'string_value': None, 'int_value': 1, 'float_value': None, 'double_value': None}}, {'key': 'engaged_session_event', 'value': {'string_value': None, 'int_value': 1, 'float_value': None, 'double_value': None}}], None, None, 777008367, None, None, 

In [56]:
job = client.get_job('bquxjob_72bb18f5_18ab14e942e') # Job ID inserted based on the query results selected to explore
print(job.query)

SELECT * FROM `analytics-399003.analytics_406290682.events_20230918` LIMIT 1000


In [113]:
dataset = "analytics_406290682"
tablename = "events_20230918"

sql = f"""
SELECT * 
FROM `analytics-399003.{dataset}.{tablename}`
LIMIT 1000
"""

In [114]:
results = client.query(sql).to_dataframe()

In [115]:
results.head()

,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,user_pseudo_id,...,geo,app_info,traffic_source,stream_id,platform,event_dimensions,ecommerce,items,collected_traffic_source,is_active_user
0,20230918,1695048275080431,sec20_timer,"[{'key': 'page_title', 'value': {'string_value...",<NA>,NaN,777008367,<NA>,None,877881965.1695048255,...,"{'city': 'Fredericia', 'country': 'Denmark', '...",None,"{'name': '(direct)', 'medium': '(none)', 'sour...",6092560544,WEB,None,None,[],None,True
1,20230918,1695048280488871,scroll_90,"[{'key': 'engaged_session_event', 'value': {'s...",<NA>,NaN,782416807,<NA>,None,877881965.1695048255,...,"{'city': 'Fredericia', 'country': 'Denmark', '...",None,"{'name': '(direct)', 'medium': '(none)', 'sour...",6092560544,WEB,None,None,[],None,True
2,20230918,1695048255149401,first_visit,"[{'key': 'ga_session_number', 'value': {'strin...",<NA>,NaN,757077337,<NA>,None,877881965.1695048255,...,"{'city': 'Fredericia', 'country': 'Denmark', '...",None,"{'name': '(direct)', 'medium': '(none)', 'sour...",6092560544,WEB,None,None,[],None,True
3,20230918,1695048255149401,session_start,"[{'key': 'ga_session_id', 'value': {'string_va...",<NA>,NaN,757077337,<NA>,None,877881965.1695048255,...,"{'city': 'Fredericia', 'country': 'Denmark', '...",None,"{'name': '(direct)', 'medium': '(none)', 'sour...",6092560544,WEB,None,None,[],None,True
4,20230918,1695048255149401,page_view,"[{'key': 'page_location', 'value': {'string_va...",<NA>,NaN,757077337,<NA>,None,877881965.1695048255,...,"{'city': 'Fredericia', 'country': 'Denmark', '...",None,"{'name': '(direct)', 'medium': '(none)', 'sour...",6092560544,WEB,None,None,[],None,True


In [116]:
columns_to_use = ['event_date', 'event_timestamp', 'event_name', 'event_params',
       'event_bundle_sequence_id',
       'user_pseudo_id',
       'user_first_touch_timestamp', 'device', 'geo',
       'traffic_source', 'stream_id', 'platform']

In [117]:
df = results.copy()

In [118]:
df = df[columns_to_use]

In [119]:
df_1 = df.copy()
df_1.head(1)

,event_date,event_timestamp,event_name,event_params,event_bundle_sequence_id,user_pseudo_id,user_first_touch_timestamp,device,geo,traffic_source,stream_id,platform
0,20230918,1695048275080431,sec20_timer,"[{'key': 'page_title', 'value': {'string_value...",777008367,877881965.1695048255,1695048255149401,"{'category': 'desktop', 'mobile_brand_name': '...","{'city': 'Fredericia', 'country': 'Denmark', '...","{'name': '(direct)', 'medium': '(none)', 'sour...",6092560544,WEB


In [120]:
e = "event_params_"
df[e + "page_title"] = None
df[e + "ga_session_id"] = None
df[e + "page_location"] = None
df[e + "session_engaged"] = None
df[e + "engagement_time_msec"] = None
df[e + "ga_session_number"] = None
df[e + "engaged_session_event"] = None

for index, row in df.iterrows():
    eventparams = row["event_params"]
    for param in eventparams:
        if param['key'] == 'page_title':
            df.at[index, "event_params_page_title"] = param['value']['string_value']

        elif param['key'] == 'ga_session_id':
            df.at[index, "event_params_ga_session_id"] = param['value']['int_value']

        elif param['key'] == 'page_location':
            df.at[index, "event_params_page_location"] = param['value']['string_value']

        elif param['key'] == 'session_engaged':
            df.at[index, "event_params_session_engaged"] = param['value']['string_value']

        elif param['key'] == 'engagement_time_msec':
            df.at[index, "event_params_engagement_time_msec"] = param['value']['int_value']

        elif param['key'] == 'ga_session_number':
            df.at[index, "event_params_ga_session_number"] = param['value']['int_value']

        elif param['key'] == 'engaged_session_event':
            df.at[index, "event_params_engaged_session_event"] = param['value']['int_value']

            break

In [121]:
df.drop(columns=["event_params", "device", "geo", "traffic_source"], inplace=True)

In [122]:
df.head(1)

,event_date,event_timestamp,event_name,event_bundle_sequence_id,user_pseudo_id,user_first_touch_timestamp,stream_id,platform,event_params_page_title,event_params_ga_session_id,event_params_page_location,event_params_session_engaged,event_params_engagement_time_msec,event_params_ga_session_number,event_params_engaged_session_event
0,20230918,1695048275080431,sec20_timer,777008367,877881965.1695048255,1695048255149401,6092560544,WEB,Viktor Byskov - Viktor Byskov,1695048254,https://viktorbyskov.dk/,1,1591,1,1


In [123]:
listofcolumns = list(df.columns)
listofcolumns

['event_date',
 'event_timestamp',
 'event_name',
 'event_bundle_sequence_id',
 'user_pseudo_id',
 'user_first_touch_timestamp',
 'stream_id',
 'platform',
 'event_params_page_title',
 'event_params_ga_session_id',
 'event_params_page_location',
 'event_params_session_engaged',
 'event_params_engagement_time_msec',
 'event_params_ga_session_number',
 'event_params_engaged_session_event']

In [124]:
capitalized_columns = [s.replace('_', ' ').title().replace(' ', '_') for s in listofcolumns]

In [125]:
capitalized_columns

['Event_Date',
 'Event_Timestamp',
 'Event_Name',
 'Event_Bundle_Sequence_Id',
 'User_Pseudo_Id',
 'User_First_Touch_Timestamp',
 'Stream_Id',
 'Platform',
 'Event_Params_Page_Title',
 'Event_Params_Ga_Session_Id',
 'Event_Params_Page_Location',
 'Event_Params_Session_Engaged',
 'Event_Params_Engagement_Time_Msec',
 'Event_Params_Ga_Session_Number',
 'Event_Params_Engaged_Session_Event']

In [126]:
df.to_csv(f"cleaned_{tablename}.csv", index=False)

In [35]:
dataset_id = "analytics-399003.analytics_406290682"
tables = client.list_tables(dataset_id)

In [36]:
print("Tables contained in '{}':".format(dataset_id))
for table in tables:
    print("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))

Tables contained in 'analytics-399003.analytics_406290682':
analytics-399003.analytics_406290682.events_20230914
analytics-399003.analytics_406290682.events_20230915
analytics-399003.analytics_406290682.events_20230916
analytics-399003.analytics_406290682.events_20230917
analytics-399003.analytics_406290682.events_20230918
analytics-399003.analytics_406290682.pseudonymous_users_20230914
analytics-399003.analytics_406290682.pseudonymous_users_20230915
analytics-399003.analytics_406290682.pseudonymous_users_20230916
analytics-399003.analytics_406290682.pseudonymous_users_20230917
analytics-399003.analytics_406290682.pseudonymous_users_20230918


In [138]:
from datetime import date
today = str(date.today().strftime("%Y%d%m"))
print(today)

20232209
